In [1]:
import os
import pandas as pd
from datetime import datetime

files = {'active': './data/active/',
        'gainers': './dada/gainers/',
        'losers': './data/losers/'}

def read_today_data(df_name, path):
    now = str(datetime.now())
    search_string = now[0:10].replace("-", "")
    for filename in os.listdir(path):
        if search_string in filename:
            file_path = os.path.join(path, filename)
            df =pd.read_csv()
            globals()[df_name] = pd.read_csv(file_path)

files = {'active': './data/active/',
        'gainers': './data/gainers/',
        'losers': './data/losers/'}

for k, v in files.items():
    read_today_data(k,v)

In [73]:
def filter_df(df, condition_dict, match='any'):
    mask = None
    for column, condition in condition_dict.items():
        column_mask = condition(df[column])
        if mask is None:
            mask = column_mask
        elif match == 'any':
            mask = mask | column_mask
        elif match == 'all':
            mask = mask & column_mask
    return df[mask]


condition_dict = {'Price (Intraday)': lambda x: x < 10, 
                  '% Change': lambda x: x > 10, 
                  'Market Cap(Billions)': lambda x: x < 10}

filtered_df = filter_df(gainers, condition_dict, match='all')
filtered_df.head(20)


,Symbol,Name,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month),Market Cap(Billions),PE Ratio (TTM)
0,BNDSY,"Banco de Sabadell, S.A.",2.58,0.49,23.44,30093.0,2365.0,8.677,9.21
1,CVT,Cvent Holding Corp.,8.07,1.50,22.83,538858.0,487739.0,3.938,NaN
4,GETY,"Getty Images Holdings, Inc.",6.36,0.92,16.91,1497000.0,333075.0,2.062,53.00


In [74]:
import pandas as pd

def filter_dataframe(df, conditions, match_all=False, compare_columns=False):
    if compare_columns:
        mask = (df[conditions[0][0]] - df[conditions[0][1]]).abs() < conditions[0][2]
    else:
        mask = df[conditions[0][0]] < conditions[0][2]
        
    for i in range(1, len(conditions)):
        if compare_columns:
            new_mask = (df[conditions[i][0]] - df[conditions[i][1]]).abs() < conditions[i][2]
        else:
            new_mask = df[conditions[i][0]] < conditions[i][2]
        
        if match_all:
            mask = mask & new_mask
        else:
            mask = mask | new_mask
    
    return df[mask]

data = {'col1': [1, 2, 3, 4],
        'col2': [10, 20, 30, 40],
        'col3': [100, 200, 300, 400]}

df = pd.DataFrame(data)

# filter the DataFrame where the value in col2 is less than 25
conditions = [('col2', 25)]
filtered_df = filter_dataframe(df, conditions)
print(filtered_df)

# filter the DataFrame where the value in col2 is less than 25 and the value in col3 is less than 250
conditions = [('col2', 25), ('col3', 250)]
filtered_df = filter_dataframe(df, conditions, match_all=True)
print(filtered_df)

# filter the DataFrame where the absolute difference between col1 and col2 is less than 5
conditions = [('col1', 'col2', 5), ('col2', 25)]
filtered_df = filter_dataframe(df, conditions, compare_columns=True)
print(filtered_df)


IndexError: tuple index out of range

In [85]:
data = {'col1': [1, 2, 3, 4],
        'col2': [10, 20, 30, 40],
        'col3': [100, 200, 300, 400]}

df = pd.DataFrame(data)

conditions = [('col2', 25), ('col3', 250)]

conditions[0][0] - df[conditions[1][1]].abs() 

KeyError: 250

In [30]:
# Importing Functions
import os
import json
import time
import requests
import pandas as pd

# Alpha vintage documentation:
# https://www.alphavantage.co/documentation/#fundamentals

# Assigning Api Key 
api_key = 'OWBLXVL83U05HM1N'

# Update_exesti  data
tech = ["AAPL", "MSFT", "GOOG", "AMZN", "META","BABA", "ASML", "AVGO", "ORCL", "ADBE", "TXN"]
semi_conductor = ["NVDA", "TSM", "ASML", "QCOM", "INTC", "AMD", "MU"]
Automakers = ["TSLA", "F", "GM", "LCID", "RIVN"]

watch_list = {"tech": tech,"semi_conductor": semi_conductor, "Automakers": Automakers}

ibm = ["IBM"]
watch_list = {"ibm": ibm}
api_key = 'DEMO'

# Setting default working directory for this script
os.chdir("C:\\Users\\navee\\Desktop\\Stock_prediction")


# Creating watch list with sectors
with open('./data/watch_list.json', 'r') as f:
    watch_list = json.load(f)


def get_fundamentals(watch_list = watch_list, api_key=api_key, last_sector = 'Automakers'):
    """
    1. This function creates csv files with fundamental data for all the tickers in `watch_list` dictionary
        - fundamental data includes:
            - Company overview
            - Earning reports
            - income statements
            - cash flows
            - balance sheets
    2. Inputs: 
        - `watch_list`: A dictionary with lists, each list have tickers belonging to different sector
        - `api_key`: API key to request data from alpha vintage 
        - `last_sector`: last sector in the `watch_list` dictionary
    """
    # Assigning last ticker value in watch list to `last_ticker``
    last_ticker = watch_list[last_sector][-1]
    
    for sector, tickers in watch_list.items():
        for ticker in tickers:
    
            # overview
            fnc = 'OVERVIEW'
            print(fnc)
            print(ticker)
            url = str("https://www.alphavantage.co/query?function="+fnc+"&symbol="+ticker+"&apikey="+api_key)
            temp = requests.get(url)
            temp = temp.json()

            # creating directories for sectors and tickers if they do not exist
            directory = str("./data/fundamentals/"+sector+"/"+ticker)
            os.makedirs(directory, exist_ok=True)
            
            # Saving dictionary to json file
            with open(str("./data/fundamentals/"+sector+"/"+ticker+"/"+ticker+"_overview.json"), 'w') as f:
                json.dump(temp, f, indent=4)
            

            # balance_sheet
            fnc = 'BALANCE_SHEET'    
            print(fnc)
            print(ticker)
            url = str("https://www.alphavantage.co/query?function="+fnc+"&symbol="+ticker+"&apikey="+api_key)
            temp = requests.get(url)
            temp = temp.json()

            # Converting json data to dataframe and saving into csv files
            quarter_reports = pd.DataFrame(data=temp['quarterlyReports'])
            quarter_reports['report_type'] = 'Quarterly'

            annal_reports = pd.DataFrame(data = temp['annualReports'])
            annal_reports['report_type'] = 'Annal'
            temp = pd.concat([annal_reports,quarter_reports])
            temp.to_csv(str("./data/fundamentals/"+sector+"/"+ticker+"/"+ticker+"_balance_sheet.csv"), index=False)
            

            # INCOME_STATEMENT
            # Fetching data
            fnc = 'INCOME_STATEMENT'
            url = str("https://www.alphavantage.co/query?function="+fnc+"&symbol="+ticker+"&apikey="+api_key)
            temp = requests.get(url)
            temp = temp.json()

            quarter_reports = pd.DataFrame(data=temp['quarterlyReports'])
            quarter_reports['report_type'] = 'quarterly'

            annal_reports = pd.DataFrame(data = temp['annualReports'])
            annal_reports['report_type'] = 'annual'
            temp = pd.concat([annal_reports,quarter_reports])
            temp.to_csv(str("./data/fundamentals/"+sector+"/"+ticker+"/"+ticker+"_income_statement.csv"), index=False)
            

            # Cash_flow
            fnc = 'CASH_FLOW'   
            url = str("https://www.alphavantage.co/query?function="+fnc+"&symbol="+ticker+"&apikey="+api_key)
            temp = requests.get(url)
            temp = temp.json()

            quarter_reports = pd.DataFrame(data=temp['quarterlyReports'])
            quarter_reports['report_type'] = 'Quarterly'

            annal_reports = pd.DataFrame(data = temp['annualReports'])
            annal_reports['report_type'] = 'Annal'
            temp = pd.concat([annal_reports,quarter_reports])
            temp.to_csv(str("./data/fundamentals/"+sector+"/"+ticker+"/"+ticker+"_cash_flow.csv"), index=False)
            
            
            # Earnings
            fnc = 'EARNINGS'
            url = str("https://www.alphavantage.co/query?function="+fnc+"&symbol="+ticker+"&apikey="+api_key)
            temp = requests.get(url)
            temp = temp.json()

            quatre_earnings = pd.DataFrame(data = temp['quarterlyEarnings'])
            quatre_earnings.to_csv(str("./data/fundamentals/"+sector+"/"+ticker+"/"+ticker+"_quarterly_earnings.csv"), index=False)
            annal_earnings = pd.DataFrame(data = temp['annualEarnings'])
            annal_earnings.to_csv(str("./data/fundamentals/"+sector+"/"+ticker+"/"+ticker+"_annal_earnings.csv"), index=False)
            
            if ticker == last_ticker:
                break
            time.sleep(61)


get_fundamentals()

OVERVIEW
AAPL
BALANCE_SHEET
AAPL
OVERVIEW
MSFT
BALANCE_SHEET
MSFT
OVERVIEW
GOOG
BALANCE_SHEET
GOOG
OVERVIEW
AMZN
BALANCE_SHEET
AMZN
OVERVIEW
META
BALANCE_SHEET
META
OVERVIEW
BABA
BALANCE_SHEET
BABA
OVERVIEW
ASML
BALANCE_SHEET
ASML
OVERVIEW
AVGO
BALANCE_SHEET
AVGO
OVERVIEW
ORCL
BALANCE_SHEET
ORCL
OVERVIEW
ADBE
BALANCE_SHEET
ADBE
OVERVIEW
TXN
BALANCE_SHEET
TXN
OVERVIEW
IBM
BALANCE_SHEET
IBM
OVERVIEW
NVDA
BALANCE_SHEET
NVDA
OVERVIEW
TSM
BALANCE_SHEET
TSM
OVERVIEW
ASML
BALANCE_SHEET
ASML
OVERVIEW
QCOM
BALANCE_SHEET
QCOM
OVERVIEW
INTC
BALANCE_SHEET
INTC
OVERVIEW
AMD
BALANCE_SHEET
AMD
OVERVIEW
MU
BALANCE_SHEET
MU
OVERVIEW
TSLA
BALANCE_SHEET
TSLA
OVERVIEW
F
BALANCE_SHEET
F
OVERVIEW
GM
BALANCE_SHEET
GM
OVERVIEW
LCID
BALANCE_SHEET
LCID
OVERVIEW
RIVN
BALANCE_SHEET
RIVN


In [28]:
time.sleep(61)

In [21]:

# Converting json data to dataframe and saving into csv files
print(ticker)
quarter_reports = pd.DataFrame(data=temp['quarterlyReports'])
quarter_reports['report_type'] = 'Quarterly'

annal_reports = pd.DataFrame(data = temp['annualReports'])
annal_reports['report_type'] = 'Annal'
temp = pd.concat([annal_reports,quarter_reports])
temp.head()


AAPL


,fiscalDateEnding,reportedCurrency,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,...,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding,report_type
0,2022-09-30,USD,352755000000,135405000000,23646000000,48304000000,4946000000,60932000000,217350000000,42117000000,...,98959000000,233256000000,60845000000,49142000000,50672000000,None,-3068000000,64849000000,15943425000,Annal
1,2021-09-30,USD,351002000000,134836000000,34940000000,62639000000,6580000000,51506000000,216166000000,39440000000,...,109106000000,241783000000,47493000000,53325000000,63090000000,None,5562000000,57365000000,16426786000,Annal
2,2020-09-30,USD,323888000000,143713000000,38016000000,90943000000,4061000000,37445000000,180175000000,36766000000,...,98667000000,216860000000,42684000000,54490000000,65339000000,None,14966000000,50779000000,16976763000,Annal
3,2019-09-30,USD,338516000000,162819000000,48844000000,100557000000,4106000000,45804000000,175697000000,37378000000,...,91807000000,209135000000,37720000000,50503000000,90488000000,None,45898000000,45174000000,4443236000,Annal
4,2018-09-30,USD,365725000000,131339000000,25913000000,25913000000,3956000000,48995000000,234386000000,41304000000,...,93735000000,220132000000,33327000000,48914000000,107147000000,None,70400000000,40201000000,4754986000,Annal


In [17]:
temp

{}